In [1]:
import os
import argparse
import logging
logging.basicConfig(level=logging.INFO)
from common import find_mxnet, data, fit
from common.util import download_file
import mxnet as mx
import numpy as np
from mxnet.symbol import *

In [2]:
def download_cifar100():
    data_dir="/efs/data/cifar-100-mxnet"
    #data_dir="/data/cifar-100-mxnet"
    fnames = (os.path.join(data_dir, "train.rec"),
              os.path.join(data_dir, "test.rec"))
    return fnames

In [3]:
(train_fname, val_fname) = download_cifar100()

# parse args
parser = argparse.ArgumentParser(description="train cifar100",
                                 formatter_class=argparse.ArgumentDefaultsHelpFormatter)
fit.add_fit_args(parser)
data.add_data_args(parser)
data.add_data_aug_args(parser)
data.set_data_aug_level(parser, 2)

num_unsup_examples = 50000
subset_factor = 5

parser.set_defaults(
    # data
    data_train     = train_fname,
    data_val       = val_fname,
    num_classes    = 100,
    image_shape    = '3,28,28',
 #   model_prefix   = '/efs/checkpoints/inceptionStandard',
    pad_size       = 4,
    # train
    batch_size     = 128,
    num_epochs     = 120,
    lr_step_epochs = '80,120',
    gpus           = '7',
    disp_batches   = 10,
    num_examples   = num_unsup_examples / subset_factor, 
    wd             = 1e-4,
    lr             = .002,
    lr_factor      = .33,
    optimizer      = 'adam'
)

args = parser.parse_args("")
unsup_multiplier = 1
labeled_per_class = 500 / subset_factor
sample_seed = 47
val_interval = 1
num_sup_examples = labeled_per_class * args.num_classes
sup_batch_size = args.batch_size

In [4]:
def get_rec_iter(args, get_val=True, devide_by=1, kv=None, shuffle=True):
    image_shape = tuple([int(l) for l in args.image_shape.split(',')])
    dtype = np.float32;
    if kv:
        (rank, nworker) = (kv.rank, kv.num_workers)
    else:
        (rank, nworker) = (0, 1)
    rgb_mean = [float(i) for i in args.rgb_mean.split(',')]
    train = mx.io.ImageRecordIter(
        path_imgrec         = args.data_train,
        label_width         = 1,
        mean_r              = rgb_mean[0],
        mean_g              = rgb_mean[1],
        mean_b              = rgb_mean[2],
        data_name           = 'dataSup',
        label_name          = 'labelSup',
        data_shape          = image_shape,
        batch_size          = args.batch_size,
        rand_crop           = args.random_crop,
        max_random_scale    = args.max_random_scale,
        pad                 = args.pad_size,
        fill_value          = 127,
        min_random_scale    = args.min_random_scale,
        max_aspect_ratio    = args.max_random_aspect_ratio,
        random_h            = args.max_random_h,
        random_s            = args.max_random_s,
        random_l            = args.max_random_l,
        max_rotate_angle    = args.max_random_rotate_angle,
        max_shear_ratio     = args.max_random_shear_ratio,
        rand_mirror         = args.random_mirror,
        preprocess_threads  = args.data_nthreads,
        shuffle             = shuffle,
        num_parts           = nworker * devide_by, #only take subset if we're in the supervised part
        part_index          = rank * devide_by)  # from every 'nworker' subset, take the first part
    if args.data_val is None or not get_val:
        return (train, None)
    val = mx.io.ImageRecordIter(
        path_imgrec         = args.data_val,
        label_width         = 1,
        mean_r              = rgb_mean[0],
        mean_g              = rgb_mean[1],
        mean_b              = rgb_mean[2],
        data_name           = 'data',
        label_name          = 'softmax_label',
        batch_size          = args.batch_size,
        data_shape          = image_shape,
        preprocess_threads  = args.data_nthreads,
        rand_crop           = False,
        rand_mirror         = False,
        num_parts           = nworker,
        part_index          = rank)
    return (train, val)

In [5]:
from common.multi_iterator import Multi_iterator
from common.data import get_partial_rec_iter

(train_sup, val) = get_partial_rec_iter(args, get_val=True, devide_by=5, shuffle=True)
(train_unsup, _) = get_partial_rec_iter(args, get_val=False, devide_by=1, shuffle=True)
    
train = Multi_iterator(train_sup, train_unsup, unsup_multiplier, num_unsup_examples, num_sup_examples)

5  times more unsup data than sup data


In [6]:
train.next().data

[<NDArray 128x3x28x28 @cpu(0)>, <NDArray 128x3x28x28 @cpu(0)>]

In [7]:
from symbols import inception_cifar as inception
from common.lba import compute_semisup_loss, logit_loss

t_nb = sup_batch_size * unsup_multiplier

def build():
    dataSup = mx.symbol.Variable(name="dataSup")
    labelSup = mx.symbol.Variable(name='labelSup')
    overall_loss = []
    
    if unsup_multiplier >= 1:
        dataUnsup = []
        for i in range(unsup_multiplier):
            dataUnsup.append(Variable(name="dataUnsup"+str(i)))

        # concat data, feed both through the network
        # then split it up again
        data = concat(dataSup, *dataUnsup, dim=0)

        embeddings = inception.build_embeddings(data, nembeddings=256)
        splitted = split(embeddings, num_outputs=(unsup_multiplier+1), axis=0, name='split')

        supEmbeddings = splitted[0]
        
        unsupEmbeddings = []
        for i in range(unsup_multiplier):
            unsupEmbeddings.append(splitted[1+i])
    
        unsupEmbeddings = concat(*unsupEmbeddings, dim=0)
            
        (walker_loss, visit_loss) = compute_semisup_loss(supEmbeddings, unsupEmbeddings, labelSup, t_nb, 
                                                     walker_weight=1.0, visit_weight=0.7)
        overall_loss = [walker_loss, visit_loss]
        
    else:
        supEmbeddings = inception.build_embeddings(dataSup, nembeddings=256)
        
    overall_loss = [logit_loss(supEmbeddings, labelSup, 100)] + overall_loss
    
    return Group(overall_loss)

In [8]:
class Multi_Accuracy(mx.metric.EvalMetric):
    """Calculate accuracies of multi label"""

    def __init__(self, num=None):
        super(Multi_Accuracy, self).__init__('multi-accuracy', num)

    def update(self, labels, preds):

        #for i in range(len(preds)):
        for i in range(1):
            pred_label = mx.nd.argmax_channel(preds[i]).asnumpy().astype('int32')
            label = labels[0].asnumpy().astype('int32')

            #mx.metric.check_label_shapes(label, pred_label)
            
            #print((pred_label.flat == label.flat).sum())
            #print(len(pred_label.flat))

            
            self.sum_metric[i] += (pred_label.flat == label.flat).sum()
            self.num_inst[i] += len(pred_label.flat)

In [9]:
# evaluation metrices
eval_metrics = Multi_Accuracy(num= 3 if unsup_multiplier >= 1 else 1)
                    
def fit_model(args, network, data, **kwargs):
    """
    train a model
    args : argparse returns
    network : the symbol definition of the nerual network
    data_loader : function that returns the train and val data iterators
    """
    # kvstore
    kv = mx.kvstore.create(args.kv_store)

    # logging
    head = '%(asctime)-15s Node[' + str(kv.rank) + '] %(message)s'
    logging.basicConfig(level=logging.DEBUG, format=head)
    logging.info('start with arguments %s', args)
    batch_size = sup_batch_size * (unsup_multiplier + 1)

    # data iterators
    (train, val) = data
    if args.test_io:
        tic = time.time()
        for i, batch in enumerate(train):
            for j in batch.data:
                j.wait_to_read()
            if (i+1) % args.disp_batches == 0:
                logging.info('Batch [%d]\tSpeed: %.2f samples/sec' % (
                    i, args.disp_batches*batch_size/(time.time()-tic)))
                tic = time.time()
        return

    # load model
    if 'arg_params' in kwargs and 'aux_params' in kwargs:
        arg_params = kwargs['arg_params']
        aux_params = kwargs['aux_params']
    else:
        sym, arg_params, aux_params = fit._load_model(args, kv.rank)
        if sym is not None:
            assert sym.tojson() == network.tojson()

    # save model
    checkpoint = fit._save_model(args, kv.rank)

    # devices for training
    devs = mx.cpu() if args.gpus is None or args.gpus is '' else [
        mx.gpu(int(i)) for i in args.gpus.split(',')]

    # learning rate
    lr, lr_scheduler = fit._get_lr_scheduler(args, kv)

    data_names = ['dataSup'] + ['dataUnsup'+str(i) for i in range(unsup_multiplier)]
        
    # create model
    model = mx.mod.Module(
        context       = devs,
        symbol        = network,
        label_names   = ['labelSup'],
        data_names   = data_names
    )
    
    def validate_model(epoch, *args):
        if epoch % val_interval != 0: 
            return
        res = model.score(val, eval_metrics)
        #TODO: pull this into default
        for name, value in res:
            print('Epoch[%d] Validation-%s=%f' % (epoch, name, value))

    
    #print(model.label_names)
    lr_scheduler  = lr_scheduler
    optimizer_params = {
            'learning_rate': lr,
            #'momentum' : args.mom,
            'wd' : args.wd,
            #'rescale_grad': 1,
            'lr_scheduler': lr_scheduler}

    #monitor = mx.mon.Monitor(interval=1000, pattern='.*aba_backward.*') 
    monitor = mx.mon.Monitor(interval=1000, pattern='.*') 
    
    # callbacks that run after each batch
    batch_end_callbacks = [mx.callback.Speedometer(batch_size, args.disp_batches)]
    
    epoch_end_callbacks = validate_model

    # run
    model.fit(train,
        begin_epoch        = args.load_epoch if args.load_epoch else 0,
        num_epoch          = args.num_epochs,
        #eval_data          = val,
        eval_metric        = eval_metrics,
        kvstore            = kv,
        optimizer          = args.optimizer,
        optimizer_params   = optimizer_params,
        #initializer        = initializer,
        arg_params         = arg_params,
        aux_params         = aux_params,
        batch_end_callback = batch_end_callbacks,
        epoch_end_callback = epoch_end_callbacks,
        allow_missing      = True
        #monitor            = monitor
             )
    return model

In [ ]:
train_sup.reset()
train_unsup.reset()

net = build()

model = fit_model(args, net, (train, val))

INFO:root:start with arguments Namespace(batch_size=128, benchmark=0, data_nthreads=4, data_train='/efs/data/cifar-100-mxnet/train.rec', data_val='/efs/data/cifar-100-mxnet/test.rec', disp_batches=10, dtype='float32', gpus='7', image_shape='3,28,28', kv_store='device', load_epoch=None, lr=0.002, lr_factor=0.33, lr_step_epochs='80,120', max_random_aspect_ratio=0, max_random_h=36, max_random_l=50, max_random_rotate_angle=0, max_random_s=50, max_random_scale=1, max_random_shear_ratio=0, min_random_scale=1, model_prefix=None, mom=0.9, monitor=0, network=None, num_classes=100, num_epochs=120, num_examples=10000.0, num_layers=None, optimizer='adam', pad_size=4, random_crop=1, random_mirror=1, rgb_mean='123.68,116.779,103.939', test_io=0, top_k=0, wd=0.0001)


[6250.0, 9375.0]


INFO:root:Epoch[0] Batch [10]	Speed: 123.42 samples/sec	Train-multi-accuracy_0=0.008523
INFO:root:Epoch[0] Batch [10]	Speed: 123.42 samples/sec	Train-multi-accuracy_1=nan
INFO:root:Epoch[0] Batch [10]	Speed: 123.42 samples/sec	Train-multi-accuracy_2=nan
INFO:root:Epoch[0] Batch [20]	Speed: 123.13 samples/sec	Train-multi-accuracy_0=0.010156
INFO:root:Epoch[0] Batch [20]	Speed: 123.13 samples/sec	Train-multi-accuracy_1=nan
INFO:root:Epoch[0] Batch [20]	Speed: 123.13 samples/sec	Train-multi-accuracy_2=nan
INFO:root:Epoch[0] Batch [30]	Speed: 122.70 samples/sec	Train-multi-accuracy_0=0.013281
INFO:root:Epoch[0] Batch [30]	Speed: 122.70 samples/sec	Train-multi-accuracy_1=nan
INFO:root:Epoch[0] Batch [30]	Speed: 122.70 samples/sec	Train-multi-accuracy_2=nan
INFO:root:Epoch[0] Batch [40]	Speed: 122.35 samples/sec	Train-multi-accuracy_0=0.012500
INFO:root:Epoch[0] Batch [40]	Speed: 122.35 samples/sec	Train-multi-accuracy_1=nan
INFO:root:Epoch[0] Batch [40]	Speed: 122.35 samples/sec	Train-multi

Epoch[0] Validation-multi-accuracy_0=0.011472
Epoch[0] Validation-multi-accuracy_1=nan
Epoch[0] Validation-multi-accuracy_2=nan


INFO:root:Epoch[1] Batch [10]	Speed: 121.70 samples/sec	Train-multi-accuracy_0=0.021307
INFO:root:Epoch[1] Batch [10]	Speed: 121.70 samples/sec	Train-multi-accuracy_1=nan
INFO:root:Epoch[1] Batch [10]	Speed: 121.70 samples/sec	Train-multi-accuracy_2=nan
INFO:root:Epoch[1] Batch [20]	Speed: 121.78 samples/sec	Train-multi-accuracy_0=0.012500
INFO:root:Epoch[1] Batch [20]	Speed: 121.78 samples/sec	Train-multi-accuracy_1=nan
INFO:root:Epoch[1] Batch [20]	Speed: 121.78 samples/sec	Train-multi-accuracy_2=nan
INFO:root:Epoch[1] Batch [30]	Speed: 121.62 samples/sec	Train-multi-accuracy_0=0.012500
INFO:root:Epoch[1] Batch [30]	Speed: 121.62 samples/sec	Train-multi-accuracy_1=nan
INFO:root:Epoch[1] Batch [30]	Speed: 121.62 samples/sec	Train-multi-accuracy_2=nan
INFO:root:Epoch[1] Batch [40]	Speed: 121.76 samples/sec	Train-multi-accuracy_0=0.010156
INFO:root:Epoch[1] Batch [40]	Speed: 121.76 samples/sec	Train-multi-accuracy_1=nan
INFO:root:Epoch[1] Batch [40]	Speed: 121.76 samples/sec	Train-multi

...

INFO:root:Epoch[59] Batch [10]	Speed: 119.85 samples/sec	Train-multi-accuracy_0=0.034091
INFO:root:Epoch[59] Batch [10]	Speed: 119.85 samples/sec	Train-multi-accuracy_1=nan
INFO:root:Epoch[59] Batch [10]	Speed: 119.85 samples/sec	Train-multi-accuracy_2=nan
INFO:root:Epoch[59] Batch [20]	Speed: 119.81 samples/sec	Train-multi-accuracy_0=0.036719
INFO:root:Epoch[59] Batch [20]	Speed: 119.81 samples/sec	Train-multi-accuracy_1=nan
INFO:root:Epoch[59] Batch [20]	Speed: 119.81 samples/sec	Train-multi-accuracy_2=nan
INFO:root:Epoch[59] Batch [30]	Speed: 119.77 samples/sec	Train-multi-accuracy_0=0.028906
INFO:root:Epoch[59] Batch [30]	Speed: 119.77 samples/sec	Train-multi-accuracy_1=nan
INFO:root:Epoch[59] Batch [30]	Speed: 119.77 samples/sec	Train-multi-accuracy_2=nan
INFO:root:Epoch[59] Batch [40]	Speed: 119.71 samples/sec	Train-multi-accuracy_0=0.033594
INFO:root:Epoch[59] Batch [40]	Speed: 119.71 samples/sec	Train-multi-accuracy_1=nan
INFO:root:Epoch[59] Batch [40]	Speed: 119.71 samples/sec

Epoch[60] Validation-multi-accuracy_0=0.039864
Epoch[60] Validation-multi-accuracy_1=nan
Epoch[60] Validation-multi-accuracy_2=nan


INFO:root:Epoch[61] Batch [10]	Speed: 120.13 samples/sec	Train-multi-accuracy_0=0.034091
INFO:root:Epoch[61] Batch [10]	Speed: 120.13 samples/sec	Train-multi-accuracy_1=nan
INFO:root:Epoch[61] Batch [10]	Speed: 120.13 samples/sec	Train-multi-accuracy_2=nan
INFO:root:Epoch[61] Batch [20]	Speed: 118.08 samples/sec	Train-multi-accuracy_0=0.039062
INFO:root:Epoch[61] Batch [20]	Speed: 118.08 samples/sec	Train-multi-accuracy_1=nan
INFO:root:Epoch[61] Batch [20]	Speed: 118.08 samples/sec	Train-multi-accuracy_2=nan
INFO:root:Epoch[61] Batch [30]	Speed: 118.31 samples/sec	Train-multi-accuracy_0=0.039062
INFO:root:Epoch[61] Batch [30]	Speed: 118.31 samples/sec	Train-multi-accuracy_1=nan
INFO:root:Epoch[61] Batch [30]	Speed: 118.31 samples/sec	Train-multi-accuracy_2=nan
INFO:root:Epoch[61] Batch [40]	Speed: 120.38 samples/sec	Train-multi-accuracy_0=0.035156
INFO:root:Epoch[61] Batch [40]	Speed: 120.38 samples/sec	Train-multi-accuracy_1=nan
INFO:root:Epoch[61] Batch [40]	Speed: 120.38 samples/sec

Epoch[61] Validation-multi-accuracy_0=0.045873
Epoch[61] Validation-multi-accuracy_1=nan
Epoch[61] Validation-multi-accuracy_2=nan


INFO:root:Epoch[62] Batch [10]	Speed: 120.24 samples/sec	Train-multi-accuracy_0=0.036932
INFO:root:Epoch[62] Batch [10]	Speed: 120.24 samples/sec	Train-multi-accuracy_1=nan
INFO:root:Epoch[62] Batch [10]	Speed: 120.24 samples/sec	Train-multi-accuracy_2=nan
INFO:root:Epoch[62] Batch [20]	Speed: 120.53 samples/sec	Train-multi-accuracy_0=0.028125
INFO:root:Epoch[62] Batch [20]	Speed: 120.53 samples/sec	Train-multi-accuracy_1=nan
INFO:root:Epoch[62] Batch [20]	Speed: 120.53 samples/sec	Train-multi-accuracy_2=nan
INFO:root:Epoch[62] Batch [30]	Speed: 120.54 samples/sec	Train-multi-accuracy_0=0.039062
INFO:root:Epoch[62] Batch [30]	Speed: 120.54 samples/sec	Train-multi-accuracy_1=nan
INFO:root:Epoch[62] Batch [30]	Speed: 120.54 samples/sec	Train-multi-accuracy_2=nan


In [ ]:
# batch 10 train 0.389, valid: 0.40